In [1]:
import os
os.environ["MODIN_ENGINE"] = "ray"

import ray
ray.init(_plasma_directory="/media/kmdr7/Seagate/MODIN", object_store_memory=10**10)

import modin.pandas as pd
from glob import glob

2021-01-24 09:26:17,889	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265
2021-01-24 09:26:17,893	INFO services.py:1651 -- object_store_memory is not verified when plasma_directory is set.


In [2]:
scenario_id = "36-1"
scenario_path = "/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware-Capture-"+scenario_id+"/"

In [4]:
!cd {scenario_path}bro && unexpand -t 4 conn.log.labeled > conn.log && sed -i '1,8d' conn.log && sed 's/ //g' -i conn.log && <<< $(cat ../../../base.log) < conn.log | sponge conn.log
!cd {scenario_path}zeek && sed -i '1,8d' flowmeter.log && sed 's/ //g' -i flowmeter.log && <<< $(cat ../../../base_zeek.log) < flowmeter.log | sponge flowmeter.log

In [3]:
conn = pd.read_csv(scenario_path + 'bro/conn.log', sep="\t", header=0)
conn.drop(conn.shape[0]-1, inplace=True)
conn = conn[["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state", "label", "detailed-label"]]
conn.head(1)

,ts,uid,id.orig_h,id.resp_h,id.resp_p,proto,service,conn_state,label,detailed-label
0,1.545396e+09,CNsbYd1o5m4QJW88Rj,192.168.1.198,185.130.215.13,57722,tcp,-,S0,Malicious,C&C-HeartBeat


In [ ]:
flow = pd.read_csv(scenario_path + 'zeek/flowmeter.log', sep="\t", header=0, index_col=False)
flow.drop(flow.shape[0]-1, inplace=True)
flow.drop("uid", axis=1, inplace=True)
flow.head(1)

In [ ]:
dataset = pd.concat([conn, flow], axis=1)
dataset.drop(["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state"], axis=1, inplace=True)
dataset.head(1)

In [ ]:
benign = dataset[dataset["label"] == "Benign"]
benign.to_csv('/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Benign/'+scenario_id+'.csv', index=False)
benign.shape

In [ ]:
malware = dataset[dataset["label"] != "Benign"]
malware.to_csv('/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware/'+scenario_id+'.csv', index=False)
malware.shape